# Fasttext

fastText is a library for efficient learning of word representations and sentence classification.

Source https://fasttext.cc/docs/en/supervised-tutorial.html 

In [34]:
import fasttext

# Quick Example

In [35]:
model = fasttext.train_supervised(input="cooking.stackexchange.txt")

Read 0M words
Number of words:  16568
Number of labels: 736
Progress: 100.0% words/sec/thread:    5830 lr:  0.000000 avg.loss: 10.072722 ETA:   0h 0m 0s


In [36]:
model.predict("How do I make a smash burger?")

(('__label__baking',), array([0.04017693]))

In [37]:
model.predict("How do I bake a red velvet cake?")

(('__label__baking',), array([0.05717409]))

In [38]:
model.predict("How can I tell if my avocado has gone bad?")

(('__label__food-safety',), array([0.02830103]))

# Try on Austen / Melville

In [40]:
import configparser
config = configparser.ConfigParser()
config.read("../../../env.ini")
data_home = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']
local_lib = config['DEFAULT']['local_lib']

In [83]:
CORPUS = pd.read_csv(f"{output_dir}/austen-melville-CORPUS.csv").set_index(['book_id','chap_id','para_num','token_num'])
LIB = pd.read_csv(f"{output_dir}/austen-melville-LIB.csv").set_index('book_id')
SENT = CORPUS.groupby(['book_id','chap_id','para_num','sent_num']).term_str\
    .apply(lambda x: ' '.join(map(str,x))).to_frame('sent_str')

In [78]:
with open("novel-train.txt", "w") as outfile:
    for idx, row in SENT.join(LIB.author, on=['book_id']).iterrows():
        line = "__label__"+row.author.split(",")[0] + ' ' + row.sent_str + "\n"
        outfile.write(line)

In [79]:
model2 = fasttext.train_supervised(input="novel-train.txt")

Read 2M words
Number of words:  40283
Number of labels: 2
Progress: 100.0% words/sec/thread:  289332 lr:  0.000000 avg.loss:  0.208182 ETA:   0h 0m 0s


In [80]:
model2.predict("I am sailing a ship")

(('__label__MELVILLE',), array([1.00001001]))

In [82]:
model2.predict("I am at Bath")

(('__label__AUSTEN',), array([0.68113929]))